In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

In [2]:
def convert_k(valor):
    "remove k in money columns and change the format to int"
    if 'k' in valor:
        return int(float(valor.replace('k', '')) * 1000)

def find_files_by_prefix(root_folder, prefix):
    matched_files = []
    for dirpath, _, filenames in os.walk(root_folder):
        for file in filenames:
            if file.startswith(prefix):
                full_path = os.path.join(dirpath, file)
                matched_files.append(full_path)
    return matched_files

def concat_from_list(file_list,encoding='iso-8859-1'):
    dataframes = []
    for file in file_list:
        try:
            df = pd.read_csv(file,encoding=encoding)
            if not df.empty:
                dataframes.append(df)
            else:
                print(f"empty file: {file}")
        except Exception as e:
            print(f"Error reading {file}: {e}")
    
    if dataframes:
        return pd.concat(dataframes, ignore_index=True)
    else:
        print("Load file fail")
        return pd.DataFrame()

def concat_csv_from_different_folders(folder="csv",prefix=None):
    if prefix is None:
        print("Add a prefix")

    file_list = find_files_by_prefix(root_folder=folder,prefix=prefix)
    df_concat = concat_from_list(file_list)
    return df_concat

def get_game_instance(value):
    last_char = value.split("-")[-1]
    return last_char

def text_to_index(df,name,number=0,extra_id=""):
    
    name_id = name+'_id'
    df[name_id] = df.index + number
    df[name_id] = name + "_" + df[name_id].astype(str) + extra_id
    return df

In [3]:
def tournament_names(folder='csv'):
    tournament_list = []
    for name in os.listdir(folder):
        path = os.path.join(folder, name)
        if os.path.isdir(path):
            tournament_list.append(name)
    return tournament_list

def region_by_id(touranment_name, region):
    for _, row in region.iterrows():
        if row['region'].lower() in touranment_name.lower():
            return row['reg_id']
    return "reg_4"

def create_draft_table(df):
    filas_transformadas = []

    for index, row in df.iterrows():
        picks = [
            row['team_1_select_1'], row['team_2_select_1'],
            row['team_1_select_2'], row['team_2_select_2'],
            row['team_1_select_3'], row['team_2_select_3'],
            row['decider']
        ]

        for pick_num, map_name in enumerate(picks, start=1):
            filas_transformadas.append({
                'team': row['team'],
                'series_id': row['series_id'],
                'order': row['order'],
                'bo': row['bo'],
                'pick': pick_num,
                'map_name': map_name,
                "match_instance" : row["match_instance"]
            })

    new_df = pd.DataFrame(filas_transformadas)
    return new_df

def first_ban(row):
    if (row['match_instance'] != "gf"):
        if (row['pick'] == 1):        
            return 1
        else:
            return 0
    elif (row['match_instance'] == "gf"):
        if (row['pick'] == 1 or row['pick'] == 2):        
            return 1
        else:
            return 0
    
    else:
         return 0

def second_ban(row):
    
        if row['pick'] == 5 and row['bo'] == 3:
            return 1
        else:
            return 0

def first_pick(row):

        if row['pick'] == 3:
            return 1
        else:
            return 0
        
   
def second_pick(row):
        if row['pick'] == 5 and row['bo'] == 5:
            return 1
        else:
            return 0
        
def decider_pick(row):
    if row['pick'] == 7:
        return 0.5
    else:
        return 0

In [4]:
#This should round only once 
region_name = ["americas","emea","china","pacific","global"]

df_region = pd.DataFrame(data=region_name, columns=["region"])
text_to_index(df_region,"reg",number=0,extra_id="")

#This should run each time a new tournament is add

tournaments = tournament_names('csv')
df_tournaments = pd.DataFrame(data=tournaments,columns=["tournament_name"])
tournament_name_list = ['Valorant Masters Toronto 2025', 'VCT 2025: Americas Stage 1',
       'VCT 2025: China Stage 1', 'VCT 2025: EMEA Stage 1',
       'VCT 2025: Pacific Stage 1']
df_tournaments["tournament_vlr_name"] = pd.Series(tournament_name_list)

df_tournaments['reg_id'] = df_tournaments['tournament_name'].apply(lambda x: region_by_id(x, df_region))

text_to_index(df_tournaments,"tour")

df_region.to_csv(path_or_buf='tables/table_region.csv',index=False,encoding='iso-8859-1')
df_tournaments.to_csv(path_or_buf='tables/table_tournament.csv',index=False,encoding='iso-8859-1')

In [5]:
df_draft_concat = concat_csv_from_different_folders(folder="csv", prefix="draft_")
df_team = pd.DataFrame(df_draft_concat["team"].unique(), columns=["team"])

In [6]:
#teams by region:
df_teams_china = pd.read_csv("csv/vct_2025_china_stage_1/draft_vct_2025_china_stage_1.csv",encoding='iso-8859-1')
df_teams_americas = pd.read_csv("csv/vct_2025_americas_stage_1/draft_vct_2025_americas_stage_1.csv",encoding='iso-8859-1')
df_teams_emea = pd.read_csv("csv/vct_2025_emea_stage_1/draft_vct_2025_emea_stage_1.csv",encoding='iso-8859-1')
df_teams_pacific = pd.read_csv("csv/vct_2025_pacific_stage_1/draft_vct_2025_pacific_stage_1.csv",encoding='iso-8859-1')

In [7]:
#this should run one by year
df_teams_amer_for_merge = pd.DataFrame(df_teams_americas["team"].unique(),columns=["team"])
df_teams_amer_for_merge["region"] = "reg_0"

df_teams_emea_for_merge = pd.DataFrame(df_teams_emea["team"].unique(),columns=["team"])
df_teams_emea_for_merge["region"] = "reg_1"

df_teams_china_for_merge = pd.DataFrame(df_teams_china["team"].unique(),columns=["team"])
df_teams_china_for_merge["region"] = "reg_2"

df_teams_apac_for_merge = pd.DataFrame(df_teams_pacific["team"].unique(),columns=["team"])
df_teams_apac_for_merge["region"] = "reg_3"

df_teams_concat = pd.concat(
    [
        df_teams_amer_for_merge,
        df_teams_emea_for_merge,
        df_teams_china_for_merge,
        df_teams_apac_for_merge,
    ],
    ignore_index=True,
)

text_to_index(df_teams_concat,"team")
#save 
df_teams_concat.to_csv(path_or_buf='tables/table_teams.csv',index=False,encoding='iso-8859-1')

In [8]:
#Update when a new player is add
df_players_stats = concat_csv_from_different_folders(folder="csv", prefix="player_stats")
df_players = df_players_stats[['player','team']].drop_duplicates(subset=['player'],ignore_index=True)
text_to_index(df_players,"player")

df_players_id = pd.merge(df_players, df_teams_concat[["team","team_id"]],how="left", left_on="team", right_on="team")

df_players_id = pd.merge(df_players_id, df_teams_concat[["team_id","region"]],how="left", left_on="team_id", right_on="team_id")

df_players_id.to_csv(path_or_buf='tables/table_players.csv',index=False,encoding='iso-8859-1')

In [9]:
#Player performance

df_round_detail = concat_csv_from_different_folders(folder="csv", prefix="round_detail")

df_filter_map = df_round_detail["map"] != "all"
df_round_detail_filter = df_round_detail[df_filter_map]
df_round_detail_filter["date-map"] = df_round_detail_filter["date"] + "-" + df_round_detail_filter["map"]

df_maps_id = pd.DataFrame(df_round_detail_filter["date-map"].unique(), columns=["date-map"])
text_to_index(df_maps_id,"map")

df_maps_id[['date', 'map']] = df_maps_id["date-map"].str.rsplit('-', n=1, expand=True)

df_match_ids = pd.DataFrame(data= df_maps_id["date"].unique(), columns=["date"])
text_to_index(df_match_ids,"series")

df_maps_id = pd.merge(df_maps_id, df_match_ids[["date","series_id"]],how="left", left_on="date", right_on="date")

df_maps_id.to_csv(path_or_buf='tables/table_maps_id.csv',index=False,encoding='iso-8859-1')

df_match_ids.to_csv(path_or_buf='tables/table_match_id.csv',index=False,encoding='iso-8859-1')

df_round_detail_filter = pd.merge(df_round_detail_filter, df_maps_id[["date-map","map_id","series_id"]],how="left", left_on="date-map", right_on="date-map")

df_round_detail_filter.to_csv(path_or_buf='tables/table_player_performance.csv',index=False,encoding='iso-8859-1')


In [10]:
#Team economy

df_team_economy = concat_csv_from_different_folders(folder="csv", prefix="team_economy")

columns_update = ["team_a_economy","team_b_economy","team_a_bank","team_b_bank"]

for column_name in columns_update:
    df_team_economy[column_name] = df_team_economy[column_name].apply(convert_k)

df_team_economy["date-map"] = df_team_economy["date"] + "-" + df_team_economy["map"]

df_team_economy = pd.merge(df_team_economy, df_maps_id[["date-map","map_id","series_id"]],how="left", left_on="date-map", right_on="date-map")

df_team_economy.to_csv(path_or_buf='tables/table_team_economy.csv',index=False,encoding='iso-8859-1')

In [12]:
#player stats

df_player_stats = concat_csv_from_different_folders(folder="csv", prefix="player_stats")

df_filter_map = df_player_stats["map"] != "all"
df_player_stats = df_player_stats[df_filter_map]

df_player_stats["date-map"] = df_player_stats["date"] + "-" + df_player_stats["map"]

df_player_stats = pd.merge(df_player_stats, df_maps_id[["date-map","map_id","series_id"]],how="left", left_on="date-map", right_on="date-map")

float_from_percentage_columns = ["kastBoth","kastT","kastCT","hsBoth","hsT","hsCT"]

def percentage_to_float(value):
    value = float(value.strip("%"))/100
    return value

# for column in float_from_percentage_columns:
#     df_player_stats[column] = df_player_stats[column].apply(percentage_to_float)

df_player_stats.to_csv(path_or_buf='tables/table_player_stats.csv',index=False,encoding='iso-8859-1')


In [13]:
#draft Not so sure about this one esta todo raro, para tener un formato que funciona en powerbi 

df_draft_concat = concat_csv_from_different_folders(folder="csv", prefix="draft_")
df_draft_concat["match_instance"] = df_draft_concat["source_url"].apply(get_game_instance)

df_draft_concat = pd.merge(df_draft_concat, df_maps_id[["date","series_id"]],how="left", left_on="date", right_on="date")

df_draft_concat_fix = create_draft_table(df_draft_concat)

df_draft_concat_fix.drop_duplicates(inplace=True, ignore_index=True)

df_draft_concat_fix['1st_ban'] = df_draft_concat_fix.apply(first_ban,axis=1)

df_draft_concat_fix['2nd_ban'] = df_draft_concat_fix.apply(second_ban,axis=1)

df_draft_concat_fix['1st_pick'] = df_draft_concat_fix.apply(first_pick, axis=1)

df_draft_concat_fix['2nd_pick'] = df_draft_concat_fix.apply(second_pick,axis=1)

df_draft_concat_fix['decider'] = df_draft_concat_fix.apply(decider_pick,axis=1)

df_draft_concat_fix.to_csv(path_or_buf='tables/table_draft.csv',index=False,encoding='iso-8859-1')

In [14]:
# round stats

df_round_concat = concat_csv_from_different_folders(folder="csv", prefix="round_detail")

#table with the maps
df_maps_name_id = pd.DataFrame(df_round_concat["map"].unique(), columns=["map"])
text_to_index(df_maps_name_id,"map")

df_maps_name_id.to_csv(path_or_buf='tables/table_maps_name_id.csv',index=False,encoding='iso-8859-1')

#continue with round stats

df_round_concat["date-map"] = df_round_concat["date"] + "-" + df_round_concat["map"]

df_round_concat = pd.merge(df_round_concat, df_maps_id[["date-map","map_id","series_id"]],how="left", left_on="date-map", right_on="date-map")

df_round_concat.to_csv(path_or_buf='tables/table_round_info.csv',index=False,encoding='iso-8859-1')


In [15]:
df_player_stats.describe()

,ratingBoth,ratingT,rating-ct,acsBoth,acsT,acsCT,killsBoth,killsT,killsCT,deadBoth,...,hsCT,fkBoth,fkT,fkCT,fdBoth,fdT,fdCT,fk-fdBoth,fk-fdT,fk-fdCT
count,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,...,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.00000,3680.000000,3680.000000
mean,0.994940,0.972867,1.019125,195.865761,192.729348,199.601087,14.680707,7.219293,7.461413,14.690761,...,0.303598,2.120380,0.980978,0.980978,2.120380,1.139402,1.139402,0.00000,-0.158424,-0.158424
std,0.336077,0.473087,0.462868,57.720775,80.562079,79.520851,5.388920,3.704571,3.785515,3.726957,...,0.146565,1.800582,1.158852,1.158852,1.673683,1.236780,1.236780,2.10592,1.536070,1.536070
min,0.030000,0.010000,0.010000,28.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-8.00000,-9.000000,-9.000000
25%,0.770000,0.660000,0.700000,155.000000,138.000000,144.000000,11.000000,5.000000,5.000000,13.000000,...,0.200000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-1.00000,-1.000000,-1.000000
50%,0.980000,0.940000,0.990000,193.000000,184.000000,194.000000,14.000000,7.000000,7.000000,15.000000,...,0.290000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.00000,0.000000,0.000000
75%,1.210000,1.240000,1.300000,233.000000,241.000000,248.000000,18.000000,10.000000,10.000000,17.000000,...,0.380000,3.000000,1.250000,1.250000,3.000000,2.000000,2.000000,1.00000,1.000000,1.000000
max,2.660000,4.070000,3.480000,471.000000,778.000000,683.000000,42.000000,24.000000,25.000000,41.000000,...,1.000000,11.000000,8.000000,8.000000,10.000000,9.000000,9.000000,9.00000,7.000000,7.000000


In [16]:
df_player_stats[df_player_stats["ratingT"] > 4]

,team,player,agent,ratingBoth,ratingT,rating-ct,acsBoth,acsT,acsCT,killsBoth,...,fdCT,fk-fdBoth,fk-fdT,fk-fdCT,map,date,event,date-map,map_id,series_id
1991,NAVI,koalanoob,Raze,1.75,4.07,1.56,359.0,683.0,333.0,16.0,...,0.0,5.0,1.0,1.0,Split,2025-04-09 13:15:00,VCT 2025: EMEA Stage 1,2025-04-09 13:15:00-Split,map_304,series_121


,team,player,agent,ratingBoth,ratingT,rating-ct,acsBoth,acsT,acsCT,killsBoth,...,fdCT,fk-fdBoth,fk-fdT,fk-fdCT,map,date,event,date-map,map_id,series_id
0,TH,RieNs,Fade,0.54,0.01,0.71,177.0,0.0,207.0,8.0,...,1.0,-1.0,-1.0,0.0,Pearl,2025-06-07 12:00:00,Valorant Masters Toronto 2025,2025-06-07 12:00:00-Pearl,map_0,series_0
1,TH,benjyfishy,Killjoy,0.53,0.81,0.48,141.0,233.0,126.0,6.0,...,1.0,1.0,1.0,0.0,Pearl,2025-06-07 12:00:00,Valorant Masters Toronto 2025,2025-06-07 12:00:00-Pearl,map_0,series_0
2,TH,Boo,Astra,0.45,0.90,0.37,105.0,263.0,79.0,5.0,...,3.0,-3.0,0.0,-3.0,Pearl,2025-06-07 12:00:00,Valorant Masters Toronto 2025,2025-06-07 12:00:00-Pearl,map_0,series_0
3,TH,Wo0t,Breach,0.41,1.05,0.30,135.0,270.0,114.0,6.0,...,2.0,-1.0,0.0,-1.0,Pearl,2025-06-07 12:00:00,Valorant Masters Toronto 2025,2025-06-07 12:00:00-Pearl,map_0,series_0
4,TH,MiniBoo,Yoru,0.20,0.01,0.27,85.0,0.0,99.0,4.0,...,1.0,0.0,0.0,0.0,Pearl,2025-06-07 12:00:00,Valorant Masters Toronto 2025,2025-06-07 12:00:00-Pearl,map_0,series_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3664,RRQ,Jemkin,Jett,1.40,1.04,1.75,273.0,256.0,291.0,24.0,...,1.0,3.0,0.0,3.0,Icebox,2025-05-11 04:00:00,VCT 2025: Pacific Stage 1,2025-05-11 04:00:00-Icebox,map_472,series_188
3665,RRQ,Monyet,Viper,1.34,0.84,1.85,292.0,226.0,359.0,25.0,...,0.0,0.0,-2.0,2.0,Icebox,2025-05-11 04:00:00,VCT 2025: Pacific Stage 1,2025-05-11 04:00:00-Icebox,map_472,series_188
3666,RRQ,crazyguy,Tejo,1.03,1.35,0.71,191.0,218.0,166.0,17.0,...,0.0,1.0,-1.0,2.0,Icebox,2025-05-11 04:00:00,VCT 2025: Pacific Stage 1,2025-05-11 04:00:00-Icebox,map_472,series_188
3667,RRQ,Kushy,Kayo,1.00,1.36,0.63,159.0,258.0,62.0,12.0,...,0.0,-1.0,-1.0,0.0,Icebox,2025-05-11 04:00:00,VCT 2025: Pacific Stage 1,2025-05-11 04:00:00-Icebox,map_472,series_188
